In [5]:
#!/usr/bin/env python3
"""
Syfoplay_Final_JSON_Kosong_Final.py
Single-file CLI music library / playlist manager dengan data persistence menggunakan JSON.
Data dimulai dari kondisi kosong.
"""

import os
import json
from collections import deque

# --- KONSTANTA FILE ---
LIBRARY_FILE = 'syfoplay_library.json'
USERS_FILE = 'syfoplay_users.json'

# -------------------------
# 1. DATA MODELS / NODES
# -------------------------
class Lagu:
    """Record node untuk lagu dengan pointer untuk kemungkinan multi-linked list."""
    def __init__(self, id_lagu, judul, artis, genre, album, durasi, file_audio="simulated_path"):
        self.id_lagu = id_lagu
        self.judul = judul
        self.artis = artis
        self.genre = genre
        self.album = album
        self.durasi = durasi
        self.file_audio = file_audio
        self.next_lagu = None
        self.next_artis = None
        self.next_genre = None

class User:
    """Tipe data pengguna: email, username, password, playlists (dict)."""
    def __init__(self, email, username, password):
        self.email = email
        self.username = username
        self.password = password
        self.playlists = {}

    def to_dict(self):
        """Mengubah objek User menjadi dictionary untuk disimpan di JSON."""
        playlists_serial = {}
        for name, playlist_ll in self.playlists.items():
            playlists_serial[name] = playlist_ll.to_list_id()
        return {
            "email": self.email,
            "username": self.username,
            "password": self.password,
            "playlists": playlists_serial
        }

# -------------------------
# 2. PLAYLIST (Singly Linked List)
# -------------------------
class NodePlaylist:
    def __init__(self, lagu):
        self.lagu = lagu
        self.next = None

class PlaylistLinkedList:
    """Singly linked list untuk playlist (head/tail untuk O(1) append)."""
    def __init__(self):
        self.head = None
        self.tail = None

    def tambah_lagu(self, lagu):
        new_node = NodePlaylist(lagu)
        if not self.head:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node

    def hapus_lagu_berdasarkan_id(self, id_lagu):
        """Hapus node berdasarkan id_lagu; kembalikan True jika dihapus."""
        if not self.head:
            return False

        if self.head.lagu.id_lagu == id_lagu:
            self.head = self.head.next
            if not self.head:
                self.tail = None
            return True

        curr = self.head
        while curr.next:
            if curr.next.lagu.id_lagu == id_lagu:
                if curr.next == self.tail:
                    self.tail = curr
                curr.next = curr.next.next
                return True
            curr = curr.next
        return False
    
    def to_list_id(self):
        """Mengembalikan list berisi id_lagu dari semua lagu di playlist."""
        ids = []
        curr = self.head
        while curr:
            ids.append(curr.lagu.id_lagu)
            curr = curr.next
        return ids

    def display(self):
        curr = self.head
        i = 1
        if not curr:
            print("\n  [Playlist kosong.]")
            return
        print("\n  No | ID     | Judul                 | Artis            | Genre")
        print("  ---|--------|-----------------------|------------------|------------")
        while curr:
            l = curr.lagu
            print(f"  {i:2} | {l.id_lagu:<6} | {l.judul[:20]:<21} | {l.artis[:15]:<16} | {l.genre[:10]:<10}")
            curr = curr.next
            i += 1

# -------------------------
# 3. LIBRARY (Singly Linked List)
# -------------------------
class LibraryMultiLinkedList:
    """Library lagu dengan penyimpanan dan pemuatan data JSON."""
    def __init__(self):
        self.head = None
        self.lagu_counter = 1

    def _generate_id(self):
        id_baru = f"L{self.lagu_counter:03}"
        self.lagu_counter += 1
        return id_baru

    def tambah_lagu(self, judul, artis, genre, album, durasi):
        id_baru = self._generate_id()
        lagu_baru = Lagu(id_baru, judul, artis, genre, album, durasi)
        lagu_baru.next_lagu = self.head
        self.head = lagu_baru
        return lagu_baru

    def display_library(self):
        curr = self.head
        i = 1
        if not curr:
            print("\n  [Library kosong.]")
            return 0
        print("\n  No | ID     | Judul                 | Artis            | Genre      | Album          | Durasi")
        print("  ---|--------|-----------------------|------------------|------------|----------------|--------")
        while curr:
            print(f"  {i:2} | {curr.id_lagu:<6} | {curr.judul[:20]:<21} | {curr.artis[:15]:<16} | {curr.genre[:10]:<10} | {curr.album[:13]:<14} | {curr.durasi:<6}")
            curr = curr.next_lagu
            i += 1
        total = i - 1
        print("\n  Total Lagu:", total)
        return total

    def cari_lagu(self, kunci, nilai):
        hasil = []
        curr = self.head
        nilai_lower = nilai.lower()
        while curr:
            if kunci == "judul" and nilai_lower in curr.judul.lower():
                hasil.append(curr)
            elif kunci == "artis" and nilai_lower in curr.artis.lower():
                hasil.append(curr)
            elif kunci == "genre" and nilai_lower in curr.genre.lower():
                hasil.append(curr)
            elif kunci == "album" and nilai_lower in curr.album.lower():
                hasil.append(curr)
            curr = curr.next_lagu
        return hasil

    def cari_lagu_berdasarkan_id(self, id_lagu):
        curr = self.head
        while curr:
            if curr.id_lagu == id_lagu:
                return curr
            curr = curr.next_lagu
        return None

    def hapus_lagu_dan_update_playlist(self, id_lagu, all_users):
        if not self.head:
            return False

        lagu_dihapus = None
        if self.head.id_lagu == id_lagu:
            lagu_dihapus = self.head
            self.head = self.head.next_lagu
        else:
            curr = self.head
            while curr.next_lagu:
                if curr.next_lagu.id_lagu == id_lagu:
                    lagu_dihapus = curr.next_lagu
                    curr.next_lagu = curr.next_lagu.next_lagu
                    break
                curr = curr.next_lagu

        if not lagu_dihapus:
            return False

        for username, user in all_users.items():
            for nama_playlist, playlist_ll in user.playlists.items():
                playlist_ll.hapus_lagu_berdasarkan_id(id_lagu)
        return True

# -------------------------
# 4. Playback queue (deque) dan history (stack)
# -------------------------
class PlaybackQueue:
    def __init__(self):
        self._items = deque()

    def enqueue(self, lagu):
        self._items.append(lagu)

    def dequeue(self):
        if self._items:
            return self._items.popleft()
        return None

    def is_empty(self):
        return len(self._items) == 0

    def __len__(self):
        return len(self._items)

class PlaybackHistory:
    def __init__(self):
        self._stack = []

    def push(self, lagu):
        self._stack.append(lagu)

    def pop(self):
        if self._stack:
            return self._stack.pop()
        return None

    def peek(self):
        return self._stack[-1] if self._stack else None

    def is_empty(self):
        return len(self._stack) == 0

    def tampilkan(self):
        if not self._stack:
            print("\n  [Histori kosong]")
            return
        print("\n  Histori Pemutaran (terbaru di atas):")
        print("  No | ID     | Judul                 | Artis")
        print("  ---|--------|-----------------------|------------------")
        for i, lagu in enumerate(reversed(self._stack), 1):
            print(f"  {i:2} | {lagu.id_lagu:<6} | {lagu.judul[:20]:<21} | {lagu.artis[:15]:<16}")

# -------------------------
# 5. APLIKASI UTAMA (CLI)
# -------------------------
class SyfoplayApp:
    def __init__(self):
        self.LIBRARY_FILE = LIBRARY_FILE
        self.USERS_FILE = USERS_FILE
        self.library = LibraryMultiLinkedList()
        self.users = {}
        self.admin_id = "admin"
        self.admin_pass = "admin123"
        self.current_user = None
        self.current_playing = None
        self.playback_queue = PlaybackQueue()
        self.history = PlaybackHistory()
        
        self._load_data()

    # --- JSON PERSISTENCE METHODS ---
    def _populate_dummy_data(self):
        # Tidak ada aksi yang dilakukan, data selalu kosong di awal.
        print("[INFO] Memulai aplikasi dengan data kosong.")
        
    def _load_data(self):
        """Membaca data library dan user dari file JSON. Jika file tidak ada, inisialisasi kosong."""
        # 1. Load Library
        try:
            with open(self.LIBRARY_FILE, 'r') as f:
                data_lib = json.load(f)
            
            self.library.lagu_counter = data_lib.get("lagu_counter", 1)
            
            head_node = None
            for item in data_lib.get("library", []): 
                lagu = Lagu(**item)
                lagu.next_lagu = head_node
                head_node = lagu
            self.library.head = head_node

        except (FileNotFoundError, json.JSONDecodeError):
            print("[INFO] File Library tidak ditemukan/rusak. Menggunakan library kosong.")

        # 2. Load Users
        try:
            with open(self.USERS_FILE, 'r') as f:
                data_users = json.load(f)

            for username, user_data in data_users.items():
                user = User(user_data["email"], user_data["username"], user_data["password"])
                
                for name, list_id_lagu in user_data.get("playlists", {}).items():
                    playlist_ll = PlaylistLinkedList()
                    for id_lagu in list_id_lagu:
                        lagu_ref = self.library.cari_lagu_berdasarkan_id(id_lagu)
                        if lagu_ref:
                            playlist_ll.tambah_lagu(lagu_ref)
                    user.playlists[name] = playlist_ll
                
                self.users[username] = user

        except (FileNotFoundError, json.JSONDecodeError):
            print("[INFO] File Users tidak ditemukan/rusak. Menggunakan user kosong.")
            
    def _save_library(self):
        """Menyimpan Library ke file JSON."""
        data_lib = {
            "lagu_counter": self.library.lagu_counter,
            "library": []
        }
        curr = self.library.head
        
        while curr:
            lagu_dict = vars(curr).copy()
            lagu_dict.pop('next_lagu', None)
            lagu_dict.pop('next_artis', None)
            lagu_dict.pop('next_genre', None)
            data_lib["library"].append(lagu_dict)
            curr = curr.next_lagu
        
        data_lib["library"].sort(key=lambda x: x['id_lagu'])
        
        try:
            with open(self.LIBRARY_FILE, 'w') as f:
                json.dump(data_lib, f, indent=2)
        except Exception as e:
            print(f"[ERROR] Gagal menyimpan library: {e}")

    def _save_users(self):
        """Menyimpan data Users (termasuk Playlists) ke file JSON."""
        data_users = {}
        for username, user in self.users.items():
            data_users[username] = user.to_dict()

        try:
            with open(self.USERS_FILE, 'w') as f:
                json.dump(data_users, f, indent=2)
        except Exception as e:
            print(f"[ERROR] Gagal menyimpan users: {e}")

    # --- UTILITY ---
    def clear_screen(self):
        os.system('cls' if os.name == 'nt' else 'clear')

    # MENU UTAMA
    def menu_utama(self):
        while True:
            self.clear_screen()
            print("====================================")
            print("🎶 SELAMAT DATANG DI SYFOPLAY (CLI) 🎶")
            print("====================================")
            print("1. Login Admin")
            print("2. Masuk sebagai User")
            print("3. Keluar Aplikasi")
            print("------------------------------------")

            pilihan = input("Masukkan pilihan (1-3): ").strip()
            if pilihan == '1':
                self.login_admin()
            elif pilihan == '2':
                self.menu_user_login_signup()
            elif pilihan == '3':
                print("\nTerima kasih telah menggunakan Syfoplay! Sampai jumpa.")
                break
            else:
                input("\nPilihan tidak valid. Tekan Enter untuk melanjutkan...")

    # ADMIN
    def login_admin(self):
        self.clear_screen()
        print("--- LOGIN ADMIN ---")
        username = input("ID Admin: ").strip()
        password = input("Password: ").strip()
        if username == self.admin_id and password == self.admin_pass:
            self.dashboard_admin()
        else:
            input("\nLogin Gagal. Tekan Enter untuk kembali ke Menu Utama...")

    def dashboard_admin(self):
        while True:
            self.clear_screen()
            print("====================================")
            print(" Hai Admin, selamat bekerja! 👋")
            print("====================================")
            self.library.display_library()
            print("\n--- MENU ADMIN ---")
            print("1. Tambah Lagu Baru")
            print("2. Update Data Lagu")
            print("3. Hapus Lagu")
            print("4. Pencarian & Putar Lagu")
            print("5. Menu Playback Queue/History")
            print("6. Keluar (Logout)")
            print("------------------------------------")

            pilihan = input("Masukkan pilihan (1-6): ").strip()

            if pilihan == '1':
                self.tambah_lagu_admin()
            elif pilihan == '2':
                self.update_lagu_admin()
            elif pilihan == '3':
                self.hapus_lagu_admin()
            elif pilihan == '4':
                self.pencarian_lagu_admin()
            elif pilihan == '5':
                self.admin_queue_history_menu()
            elif pilihan == '6':
                print("\nLogout berhasil. Kembali ke Menu Utama.")
                input("Tekan Enter...")
                break
            else:
                input("\nPilihan tidak valid. Tekan Enter untuk melanjutkan...")

    def tambah_lagu_admin(self):
        print("\n--- TAMBAH LAGU BARU ---")
        judul = input("Judul Lagu: ").strip()
        artis = input("Artis/Penyanyi: ").strip()
        genre = input("Genre: ").strip()
        album = input("Album: ").strip()
        durasi = input("Durasi (m:ss): ").strip()
        print("\n[Simulasi] Upload file audio berhasil...")
        lagu = self.library.tambah_lagu(judul, artis, genre, album, durasi)
        self._save_library()
        input(f"\n Lagu '{lagu.judul}' (ID: {lagu.id_lagu}) berhasil ditambahkan ke library! Tekan Enter...")

    def hapus_lagu_admin(self):
        total = self.library.display_library()
        if total == 0:
            input("\n[Perhatian] Library kosong. Tekan Enter untuk kembali...")
            return
        id_hapus = input("\nMasukkan ID lagu yang ingin dihapus (e.g. L001): ").strip()
        if not id_hapus:
            input("Input kosong. Tekan Enter...")
            return
        success = self.library.hapus_lagu_dan_update_playlist(id_hapus, self.users)
        if success:
            self._save_library()
            self._save_users()
            input(f"\n Lagu dengan ID '{id_hapus}' berhasil dihapus dari Library dan semua playlist user. Tekan Enter...")
        else:
            input(f"\n Lagu dengan ID '{id_hapus}' tidak ditemukan. Tekan Enter...")

    def update_lagu_admin(self):
        total = self.library.display_library()
        if total == 0:
            input("\n[Perhatian] Library kosong. Tekan Enter untuk kembali...")
            return
        id_update = input("\nMasukkan ID lagu yang ingin di-update (e.g. L001): ").strip()
        if not id_update:
            input("Input kosong. Tekan Enter...")
            return
        lagu_node = self.library.cari_lagu_berdasarkan_id(id_update)
        if not lagu_node:
            input(f"\n Lagu dengan ID '{id_update}' tidak ditemukan. Tekan Enter...")
            return

        print(f"\n--- UPDATE DATA LAGU: {lagu_node.judul} (ID: {lagu_node.id_lagu}) ---")
        new_judul = input(f"Judul baru (kosong = tidak diubah) [{lagu_node.judul}]: ").strip() or lagu_node.judul
        new_artis = input(f"Artis baru (kosong = tidak diubah) [{lagu_node.artis}]: ").strip() or lagu_node.artis
        new_genre = input(f"Genre baru (kosong = tidak diubah) [{lagu_node.genre}]: ").strip() or lagu_node.genre
        new_album = input(f"Album baru (kosong = tidak diubah) [{lagu_node.album}]: ").strip() or lagu_node.album
        new_durasi = input(f"Durasi baru (kosong = tidak diubah) [{lagu_node.durasi}]: ").strip() or lagu_node.durasi

        lagu_node.judul = new_judul
        lagu_node.artis = new_artis
        lagu_node.genre = new_genre
        lagu_node.album = new_album
        lagu_node.durasi = new_durasi

        self._save_library()
        input(f"\n✅ Lagu '{id_update}' berhasil di-update! Tekan Enter...")

    def tampilkan_hasil_pencarian(self, hasil):
        if not hasil:
            print("\n[INFO] Tidak ada lagu yang ditemukan.")
            return

        print("\n  --- HASIL PENCARIAN ---")
        print("  No | ID     | Judul                 | Artis            | Genre")
        print("  ---|--------|-----------------------|------------------|------------")
        for i, l in enumerate(hasil, 1):
            print(f"  {i:2} | {l.id_lagu:<6} | {l.judul[:20]:<21} | {l.artis[:15]:<16} | {l.genre[:10]:<10}")

    def pencarian_lagu_admin(self):
        while True:
            self.clear_screen()
            print("--- PENCARIAN LAGU ---")
            print("Cari berdasarkan: 1. Judul | 2. Artis | 3. Genre | 4. Album | 5. Kembali")
            pilihan = input("Pilih (1-5): ").strip()
            
            if pilihan == '5':
                break
            
            kunci = None
            if pilihan == '1': kunci = 'judul'
            elif pilihan == '2': kunci = 'artis'
            elif pilihan == '3': kunci = 'genre'
            elif pilihan == '4': kunci = 'album'
            else:
                input("\nPilihan tidak valid. Tekan Enter...")
                continue
            
            nilai = input(f"Masukkan kata kunci {kunci}: ").strip()
            if not nilai: continue
            
            hasil = self.library.cari_lagu(kunci, nilai)
            self.tampilkan_hasil_pencarian(hasil)
            
            if hasil:
                putar = input("\nIngin PUTAR salah satu lagu? (y/t): ").strip().lower()
                if putar == 'y':
                    self.putar_lagu_admin(hasil)
            
            input("\nTekan Enter untuk melanjutkan pencarian...")

    def putar_lagu_admin(self, list_lagu=None):
        if not list_lagu:
            total = self.library.display_library()
            if total == 0:
                input("\n[Perhatian] Library kosong. Tekan Enter untuk kembali...")
                return
            lagu_list = []
            curr = self.library.head
            while curr:
                lagu_list.append(curr)
                curr = curr.next_lagu
        else:
            lagu_list = list_lagu

        id_putar = input("\nMasukkan ID lagu yang ingin diputar: ").strip()
        lagu = next((l for l in lagu_list if l.id_lagu == id_putar), None)
        
        if lagu:
            self.play_lagu(lagu)
        else:
            input("\nID lagu tidak ditemukan. Tekan Enter...")

    def play_lagu(self, lagu):
        if self.current_playing:
            self.history.push(self.current_playing)
        self.current_playing = lagu
        print("\n====================================")
        print(f"▶️ Sedang Memutar: {lagu.judul} oleh {lagu.artis} ({lagu.durasi})")
        print("====================================")
        print("Opsi: 1. Next Queue | 2. Prev History | 3. Stop")
        pilihan = input("Pilih (1-3): ").strip()
        if pilihan == '1':
            self.next_queue()
        elif pilihan == '2':
            self.prev_history()
        elif pilihan == '3':
            self.current_playing = None
        else:
            input("Pilihan tidak valid. Tekan Enter...")


    def next_queue(self):
        if self.playback_queue.is_empty():
            print("\n[INFO] Queue kosong. Tidak ada lagu berikutnya.")
            self.current_playing = None
            input("Tekan Enter...")
        else:
            lagu_next = self.playback_queue.dequeue()
            self.play_lagu(lagu_next)

    def prev_history(self):
        if self.history.is_empty():
            print("\n[INFO] History kosong. Tidak ada lagu sebelumnya.")
            input("Tekan Enter...")
        else:
            if self.current_playing:
                self.playback_queue.enqueue(self.current_playing)
            
            lagu_prev = self.history.pop()
            self.current_playing = lagu_prev
            print("\n====================================")
            print(f"⏮️ Memutar Ulang: {lagu_prev.judul} oleh {lagu_prev.artis} ({lagu_prev.durasi})")
            print("====================================")
            input("Tekan Enter...")


    def admin_queue_history_menu(self):
        while True:
            self.clear_screen()
            print("--- MENU PLAYBACK ---")
            print(f"▶️ Sedang Diputar: {self.current_playing.judul if self.current_playing else '[Tidak Ada]'}")
            print(f"1. Lihat Queue ({len(self.playback_queue)})")
            print(f"2. Lihat History ({len(self.history._stack)})")
            print("3. Tambah Lagu ke Queue (Dari Library)")
            print("4. Kembali ke Dashboard Admin")
            pilihan = input("Pilih (1-4): ").strip()

            if pilihan == '1':
                self.tampilkan_queue()
            elif pilihan == '2':
                self.history.tampilkan()
                input("\nTekan Enter untuk melanjutkan...")
            elif pilihan == '3':
                self.tambah_ke_queue_dari_library()
            elif pilihan == '4':
                break
            else:
                input("\nPilihan tidak valid. Tekan Enter...")

    def tampilkan_queue(self):
        if self.playback_queue.is_empty():
            print("\n[INFO] Queue kosong.")
            input("Tekan Enter...")
            return
        
        print("\n--- PLAYBACK QUEUE (Next Up) ---")
        print("  No | ID     | Judul                 | Artis")
        print("  ---|--------|-----------------------|------------------")
        
        for i, lagu in enumerate(self.playback_queue._items, 1):
            print(f"  {i:2} | {lagu.id_lagu:<6} | {lagu.judul[:20]:<21} | {lagu.artis[:15]:<16}")
        
        input("\nTekan Enter untuk melanjutkan...")

    def tambah_ke_queue_dari_library(self):
        total = self.library.display_library()
        if total == 0:
            input("\n[Perhatian] Library kosong. Tekan Enter untuk kembali...")
            return
        id_tambah = input("\nMasukkan ID lagu yang ingin ditambahkan ke Queue: ").strip()
        lagu = self.library.cari_lagu_berdasarkan_id(id_tambah)
        if lagu:
            self.playback_queue.enqueue(lagu)
            input(f"\n Lagu '{lagu.judul}' berhasil ditambahkan ke Queue! Tekan Enter...")
        else:
            input("\nID lagu tidak ditemukan. Tekan Enter...")

    # USER (signup/login/dashboard)
    def menu_user_login_signup(self):
        while True:
            self.clear_screen()
            print("--- MENU USER ---")
            print("1. Login")
            print("2. Sign Up")
            print("3. Kembali ke Menu Utama")
            
            pilihan = input("Pilih (1-3): ").strip()
            if pilihan == '1':
                self.login_user()
            elif pilihan == '2':
                self.signup_user()
            elif pilihan == '3':
                break
            else:
                input("\nPilihan tidak valid. Tekan Enter...")

    def signup_user(self):
        self.clear_screen()
        print("--- SIGN UP AKUN BARU ---")
        email = input("Email: ").strip()
        username = input("Username: ").strip()
        password = input("Password: ").strip()
        verif_pass = input("Verifikasi Password: ").strip()
        if password != verif_pass:
            input("\n Peringatan: Verifikasi Password tidak sesuai! Tekan Enter...")
            return
        if username in self.users:
            input("\n Peringatan: Username sudah terdaftar! Tekan Enter...")
            return
        
        self.users[username] = User(email, username, password)
        self._save_users()
        input("\n Sign Up Berhasil! Silakan Login. Tekan Enter...")

    def login_user(self):
        self.clear_screen()
        print("--- LOGIN USER ---")
        username = input("Username: ").strip()
        password = input("Password: ").strip()
        
        user = self.users.get(username)
        if user and user.password == password:
            self.current_user = user
            self.dashboard_user()
        else:
            input("\nLogin Gagal: Username atau Password salah. Tekan Enter...")
    
    # DASHBOARD USER
    def dashboard_user(self):
        print(f"\nSelamat datang, {self.current_user.username}!")
        while True:
            self.clear_screen()
            print("====================================")
            print(f"👤 DASHBOARD {self.current_user.username.upper()} 🎧")
            print("====================================")
            print(f"▶️ Sedang Diputar: {self.current_playing.judul if self.current_playing else '[Tidak Ada]'}")
            print("\n--- MENU USER ---")
            print("1. Buat Playlist Baru")
            print("2. Akses Playlist Saya")
            print("3. Cari Lagu & Tambahkan ke Playlist")
            print("4. Lihat Katalog Artis")
            print("5. Menu Playback Queue/History")
            print("6. Keluar (Logout)")
            print("------------------------------------")

            pilihan = input("Masukkan pilihan (1-6): ").strip()

            if pilihan == '1':
                self.buat_playlist_user()
            elif pilihan == '2':
                self.menu_playlist_user()
            elif pilihan == '3':
                self.pencarian_lagu_user()
            elif pilihan == '4':
                self.menu_artis_user()
            elif pilihan == '5':
                self.admin_queue_history_menu()
            elif pilihan == '6':
                self.current_user = None
                self.current_playing = None
                self.playback_queue = PlaybackQueue()
                self.history = PlaybackHistory()
                print("\nLogout berhasil. Kembali ke Menu User.")
                input("Tekan Enter...")
                break
            else:
                input("\nPilihan tidak valid. Tekan Enter untuk melanjutkan...")

    def buat_playlist_user(self):
        print("\n--- BUAT PLAYLIST BARU ---")
        nama = input("Masukkan NAMA Playlist: ").strip()
        if not nama:
            input("Nama kosong. Tekan Enter...")
            return
        if nama in self.current_user.playlists:
            input("\n Peringatan: Nama Playlist sudah ada! Tekan Enter...")
            return
        
        self.current_user.playlists[nama] = PlaylistLinkedList()
        self._save_users()
        input(f"\n Playlist '{nama}' berhasil dibuat! Tekan Enter...")

    def pencarian_lagu_user(self):
        while True:
            self.clear_screen()
            print("--- PENCARIAN LAGU ---")
            print("Cari berdasarkan: 1. Judul | 2. Artis | 3. Genre | 4. Album | 5. Kembali")
            pilihan = input("Pilih (1-5): ").strip()
            
            if pilihan == '5':
                break
            
            kunci = None
            if pilihan == '1': kunci = 'judul'
            elif pilihan == '2': kunci = 'artis'
            elif pilihan == '3': kunci = 'genre'
            elif pilihan == '4': kunci = 'album'
            else:
                input("\nPilihan tidak valid. Tekan Enter...")
                continue
            
            nilai = input(f"Masukkan kata kunci {kunci}: ").strip()
            if not nilai: continue
            
            hasil = self.library.cari_lagu(kunci, nilai)
            self.tampilkan_hasil_pencarian(hasil)
            
            if hasil:
                tambah = input("\nIngin MENAMBAHKAN hasil pencarian ke Playlist? (y/t): ").strip().lower()
                if tambah == 'y':
                    id_tambah = input("Masukkan ID lagu yang ingin ditambahkan (e.g. L001): ").strip()
                    node = self.library.cari_lagu_berdasarkan_id(id_tambah)
                    if node:
                        self._add_lagu_to_playlist(node)
                    else:
                        input("ID tidak ditemukan. Tekan Enter...")
                
                putar = input("\nIngin PUTAR salah satu lagu? (y/t): ").strip().lower()
                if putar == 'y':
                    self.putar_lagu_admin(hasil)
            
            input("\nTekan Enter untuk melanjutkan pencarian...")

    def _add_lagu_to_playlist(self, lagu):
        if not self.current_user.playlists:
            input("\n[Perhatian] Anda belum memiliki Playlist. Buat Playlist baru terlebih dahulu. Tekan Enter...")
            return
        
        print("\n--- PILIH PLAYLIST ---")
        playlist_list = list(self.current_user.playlists.keys())
        for idx, name in enumerate(playlist_list, 1):
            print(f"  {idx}. {name}")
        try:
            pilihan_idx = int(input("Pilih nomor Playlist: ").strip()) - 1
            if 0 <= pilihan_idx < len(playlist_list):
                nama_playlist = playlist_list[pilihan_idx]
                playlist = self.current_user.playlists[nama_playlist]
                playlist.tambah_lagu(lagu)
                self._save_users()
                input(f"\n Lagu '{lagu.judul}' berhasil ditambahkan ke Playlist '{nama_playlist}'! Tekan Enter...")
            else:
                input("\n Pilihan Playlist tidak valid. Tekan Enter...")
        except ValueError:
            input("\n Masukan harus berupa angka. Tekan Enter...")

    def menu_playlist_user(self):
        while True:
            self.clear_screen()
            print("--- PLAYLIST SAYA ---")
            if not self.current_user.playlists:
                input("\n[INFO] Anda belum memiliki playlist. Tekan Enter...")
                return
            
            playlist_list = list(self.current_user.playlists.keys())
            for idx, name in enumerate(playlist_list, 1):
                print(f"  {idx}. {name}")
            print(f"  {len(playlist_list) + 1}. Kembali")

            pilihan = input("Pilih nomor Playlist atau Kembali: ").strip()
            try:
                pilihan_idx = int(pilihan) - 1
                if 0 <= pilihan_idx < len(playlist_list):
                    nama_playlist = playlist_list[pilihan_idx]
                    self.akses_playlist_terpilih(nama_playlist)
                elif pilihan_idx == len(playlist_list):
                    break
                else:
                    input("\n Pilihan tidak valid. Tekan Enter...")
            except ValueError:
                input("\n Masukan harus berupa angka. Tekan Enter...")

    def akses_playlist_terpilih(self, nama_playlist):
        playlist = self.current_user.playlists[nama_playlist]
        while True:
            self.clear_screen()
            print(f"--- PLAYLIST: {nama_playlist.upper()} ---")
            playlist.display()
            print("\nOpsi: 1. Putar Playlist | 2. Hapus Lagu | 3. Kembali")
            pilihan = input("Pilih (1-3): ").strip()
            
            if pilihan == '1':
                self.putar_lagu_playlist(playlist)
            elif pilihan == '2':
                self.hapus_lagu_playlist(playlist, nama_playlist)
            elif pilihan == '3':
                break
            else:
                input("\nPilihan tidak valid. Tekan Enter...")

    def putar_lagu_playlist(self, playlist):
        if not playlist.head:
            input("\n[Perhatian] Playlist kosong. Tekan Enter...")
            return

        print("\n--- PUTAR PLAYLIST ---")
        id_putar = input("Masukkan ID lagu yang ingin diputar: ").strip()
        
        curr = playlist.head
        lagu_mulai = None
        while curr:
            if curr.lagu.id_lagu == id_putar:
                lagu_mulai = curr.lagu
                break
            curr = curr.next
        
        if not lagu_mulai:
            input("\nID lagu tidak ditemukan dalam Playlist ini. Tekan Enter...")
            return

        self.playback_queue = PlaybackQueue()
        self.history = PlaybackHistory()
        
        curr = curr.next
        while curr:
            self.playback_queue.enqueue(curr.lagu)
            curr = curr.next

        self.play_lagu(lagu_mulai)

    def hapus_lagu_playlist(self, playlist, nama_playlist):
        if not playlist.head:
            input("\n[Perhatian] Playlist kosong, tidak ada yang bisa dihapus. Tekan Enter...")
            return
        id_hapus = input("\nMasukkan ID lagu yang ingin dihapus dari Playlist: ").strip()
        if not id_hapus:
            input("Input kosong. Tekan Enter...")
            return
        if playlist.hapus_lagu_berdasarkan_id(id_hapus):
            self._save_users()
            input(f"\n Lagu '{id_hapus}' berhasil dihapus dari Playlist '{nama_playlist}'. Tekan Enter...")
        else:
            input(f"\n Lagu dengan ID '{id_hapus}' tidak ditemukan di Playlist. Tekan Enter...")

    # ARTIST CATALOG
    def menu_artis_user(self):
        self.clear_screen()
        print("--- KATALOG ARTIS ---")
        artis_set = set()
        curr = self.library.head
        while curr:
            artis_set.add(curr.artis)
            curr = curr.next_lagu
        
        if not artis_set:
            input("\n[INFO] Library kosong. Tekan Enter...")
            return

        artis_list = sorted(list(artis_set))
        for i, artis in enumerate(artis_list, 1):
            print(f"  {i}. {artis}")
        print(f"  {len(artis_list) + 1}. Kembali")
        
        pilihan = input("\nPilih nomor Artis: ").strip()
        try:
            pilihan_idx = int(pilihan) - 1
            if 0 <= pilihan_idx < len(artis_list):
                nama_artis = artis_list[pilihan_idx]
                self.list_lagu_artis(nama_artis)
            elif pilihan_idx == len(artis_list):
                pass
            else:
                input("\nPilihan tidak valid. Tekan Enter...")
        except ValueError:
            input("\nMasukan harus berupa angka. Tekan Enter...")

    def list_lagu_artis(self, nama_artis):
        self.clear_screen()
        print(f"--- LAGU DARI ARTIS: {nama_artis.upper()} ---")
        lagu_artis = []
        curr = self.library.head
        while curr:
            if curr.artis.lower() == nama_artis.lower():
                lagu_artis.append(curr)
            curr = curr.next_lagu

        self.tampilkan_hasil_pencarian(lagu_artis)

        if not lagu_artis:
            input("\n[INFO] Tidak ada lagu ditemukan untuk artis ini. Tekan Enter...")
            return

        print("\nOpsi: 1. Putar | 2. Tambah ke Playlist | 3. Kembali")
        pilihan = input("Pilih (1-3): ").strip()

        if pilihan == '1':
            self.putar_lagu_admin(lagu_artis)
        elif pilihan == '2':
            id_tambah = input("\nMasukkan ID lagu yang ingin ditambahkan ke Playlist: ").strip()
            lagu_ditemukan = next((l for l in lagu_artis if l.id_lagu == id_tambah), None)
            if lagu_ditemukan:
                self._add_lagu_to_playlist(lagu_ditemukan)
            else:
                input("\n Lagu tidak ditemukan. Tekan Enter...")
        elif pilihan == '3':
            return
        else:
            input("\nPilihan tidak valid. Tekan Enter...")

# -------------------------
# 6. EXECUTE
# -------------------------
if __name__ == "__main__":
    app = SyfoplayApp()
    app.menu_utama()

[INFO] File Library tidak ditemukan/rusak. Menggunakan library kosong.
[INFO] File Users tidak ditemukan/rusak. Menggunakan user kosong.
🎶 SELAMAT DATANG DI SYFOPLAY (CLI) 🎶
1. Login Admin
2. Masuk sebagai User
3. Keluar Aplikasi
------------------------------------
--- LOGIN ADMIN ---
 Hai Admin, selamat bekerja! 👋

  [Library kosong.]

--- MENU ADMIN ---
1. Tambah Lagu Baru
2. Update Data Lagu
3. Hapus Lagu
4. Pencarian & Putar Lagu
5. Menu Playback Queue/History
6. Keluar (Logout)
------------------------------------

--- TAMBAH LAGU BARU ---

[Simulasi] Upload file audio berhasil...
 Hai Admin, selamat bekerja! 👋

  No | ID     | Judul                 | Artis            | Genre      | Album          | Durasi
  ---|--------|-----------------------|------------------|------------|----------------|--------
   1 | L001   | Kange                 | Dewa 19          | Pop Rock   | 19             | 5:00  

  Total Lagu: 1

--- MENU ADMIN ---
1. Tambah Lagu Baru
2. Update Data Lagu
3. Hapus